In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from shapely.geometry import Point

# Define the data
data = {
    "Institution": [
        "NASA Jet Propulsion Laboratory",
        "National Academy of Sciences",
        "Northeastern University",
        "IIASA",
        "American Geophysical Union",
        "Princeton University"
    ],
    "Location": [
        "Pasadena, CA, USA",
        "Washington, DC, USA",
        "Boston, MA, USA",
        "Vienna, Austria",
        "New Orleans, LA, USA",
        "Princeton, NJ, USA"
    ],
    "Number of People": [50, 250, 40, 25, 25000, 60]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Geocode locations
geolocator = Nominatim(user_agent="geoapi")
def geocode(location):
    try:
        loc = geolocator.geocode(location)
        return pd.Series([loc.latitude, loc.longitude])
    except:
        return pd.Series([None, None])

df[['Latitude', 'Longitude']] = df['Location'].apply(geocode)

# Drop rows with missing coordinates
df.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Create GeoDataFrame
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Load world map
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Plot
fig, ax = plt.subplots(figsize=(15, 10))
world.plot(ax=ax, color='lightgray', edgecolor='black')
gdf.plot(ax=ax, markersize=gdf['Number of People'] / 5, color='red', alpha=0.6)

# Annotate points
for x, y, label in zip(gdf.geometry.x, gdf.geometry.y, gdf['Institution']):
    ax.text(x + 0.5, y, label, fontsize=9)

plt.title("Institution Locations with Number of People")
plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

# Function to create great-circle arc between two points
def great_circle_arc(p1, p2, num_points=100):
    lon1, lat1 = np.radians(p1)
    lon2, lat2 = np.radians(p2)

    # Compute spherical interpolation
    delta = np.arccos(np.sin(lat1) * np.sin(lat2) +
                      np.cos(lat1) * np.cos(lat2) * np.cos(lon2 - lon1))
    if delta == 0:
        return np.array([p1])  # Same point

    sin_delta = np.sin(delta)
    points = []
    for f in np.linspace(0, 1, num_points):
        A = np.sin((1 - f) * delta) / sin_delta
        B = np.sin(f * delta) / sin_delta
        x = A * np.cos(lat1) * np.cos(lon1) + B * np.cos(lat2) * np.cos(lon2)
        y = A * np.cos(lat1) * np.sin(lon1) + B * np.cos(lat2) * np.sin(lon2)
        z = A * np.sin(lat1) + B * np.sin(lat2)
        lat = np.arctan2(z, np.sqrt(x**2 + y**2))
        lon = np.arctan2(y, x)
        points.append((np.degrees(lon), np.degrees(lat)))
    return np.array(points)

# Plot with arcs
fig, ax = plt.subplots(figsize=(15, 10))
world.plot(ax=ax, color='lightgray', edgecolor='black')
gdf.plot(ax=ax, markersize=gdf['Number of People'] / 5, color='green', alpha=0.7)

# Draw arcs between all points
coords = list(zip(gdf['Longitude'], gdf['Latitude']))
for i, start in enumerate(coords):
    for j, end in enumerate(coords):
        if i < j:  # Avoid duplicate and self arcs
            arc = great_circle_arc(start, end)
            ax.plot(arc[:, 0], arc[:, 1], color='blue', alpha=0.4, linewidth=1)

# Annotate points
for x, y, label in zip(gdf.geometry.x, gdf.geometry.y, gdf['Institution']):
    ax.text(x + 0.5, y, label, fontsize=9)

plt.title("Institution Locations with Connecting Arcs")
plt.axis('off')
plt.tight_layout()
plt.show()
